In [228]:
import dash
from dash import html
from dash import dcc
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data Preparation

In [229]:
import pandas as pd
import numpy as np
df=pd.read_csv("covid-vaccination-global-projections.csv")
df.tail()

,Entity,Code,Year,status
229,Wales,NaN,2021,Already above 40% vaccinated
230,Wallis and Futuna,WLF,2021,Already above 40% vaccinated
231,World,OWID_WRL,2021,Already above 40% vaccinated
232,Yemen,YEM,2021,Not on track to 40% vaccinated
233,Zimbabwe,ZWE,2021,Not on track to 40% vaccinated


In [230]:
df=df.fillna(0)

In [231]:
df.tail()

,Entity,Code,Year,status
229,Wales,0,2021,Already above 40% vaccinated
230,Wallis and Futuna,WLF,2021,Already above 40% vaccinated
231,World,OWID_WRL,2021,Already above 40% vaccinated
232,Yemen,YEM,2021,Not on track to 40% vaccinated
233,Zimbabwe,ZWE,2021,Not on track to 40% vaccinated


In [232]:
df2 = pd.read_csv("owid-covid-data (2).csv")

In [233]:
cols = ['new_cases_smoothed','new_deaths_smoothed','total_cases_per_million','new_cases_per_million','new_cases_smoothed_per_million',
 'total_deaths_per_million','new_deaths_per_million','new_deaths_smoothed_per_million','icu_patients_per_million',
 'hosp_patients_per_million','weekly_icu_admissions_per_million','weekly_hosp_admissions_per_million','new_tests_smoothed',
 'new_tests_smoothed_per_thousand','total_tests_per_thousand','new_tests_per_thousand','new_vaccinations_smoothed',
 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred',
 'total_boosters_per_hundred','new_vaccinations_smoothed_per_million','female_smokers','male_smokers',
 'excess_mortality_cumulative_per_million']
df2 = df2.drop(cols,axis=1)

In [234]:
df2["Date"] = pd.to_datetime(df2.date, format="%Y-%m-%d")

### Extract top 5 countries with fully vaccinated people

In [235]:
x_5 = df2[df2["Date"] == "2021-10-31"]
x_5 = x_5.set_index("location")
x_5 = x_5.drop(['Africa','Europe','Asia','North America','South America',"World","European Union"])
x_5 = x_5.sort_values(["people_fully_vaccinated","people_vaccinated"],ascending=False)
x_5 = x_5[:][0:5].sort_values("people_fully_vaccinated",ascending=True)
x_5

,iso_code,continent,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,hosp_patients,...,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,Date
location,,,,,,,,,,,,,,,,,,,,,
Germany,DEU,Europe,2021-10-31,4607958.0,-554.0,95735.0,1.0,NaN,1346.0,NaN,...,156.139,8.31,NaN,8.00,81.33,0.947,NaN,NaN,NaN,2021-10-31
Indonesia,IDN,Asia,2021-10-31,4244358.0,523.0,143405.0,17.0,NaN,NaN,NaN,...,342.864,6.32,64.204,1.04,71.72,0.718,NaN,NaN,NaN,2021-10-31
Japan,JPN,Asia,2021-10-31,1723559.0,232.0,18275.0,7.0,NaN,NaN,NaN,...,79.370,5.72,NaN,13.05,84.63,0.919,NaN,NaN,NaN,2021-10-31
United States,USA,North America,2021-10-31,45970785.0,17599.0,745832.0,164.0,NaN,18096.0,57618.0,...,151.089,10.79,NaN,2.77,78.86,0.926,NaN,NaN,NaN,2021-10-31
India,IND,Asia,2021-10-31,34285814.0,12514.0,458437.0,251.0,NaN,NaN,NaN,...,282.280,10.39,59.550,0.53,69.66,0.645,NaN,NaN,NaN,2021-10-31


### Extract Egypt's vaccinated people

In [236]:
x_6 = df2[df2["location"] == "Egypt"]
x_6 = x_6[(~(x_6["people_fully_vaccinated"].isna()))&(~(x_6["people_vaccinated"].isna()))]
x_E = x_6[x_6["Date"]=="2021-10-23"]

### create date list for slider

In [237]:
date_list=pd.date_range('2020-01-01','2021-10-31', freq='MS').tolist()

In [238]:
external_style = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Dash

In [239]:
app=dash.Dash(external_stylesheets=external_style)

### Header

In [240]:
header=html.Div([html.H1('Covid19-Vaccine',
                             style={'font-family':'Impact','font-size':80,'backgroundColor':'#1167b1','color':'white','textAlign':'left','marginBottom': 10,'fontWeight': 'bold', 'marginTop':10,'marginLeft': 10}),
html.H1('From Feb 2020 to Oct 2021',
                             style={'font-size':30,'backgroundColor':'#1167b1','color':'white','textAlign':'left','marginBottom': 10,'fontWeight': 'bold', 'marginTop': 0,'marginLeft': 100})],style={'marginBottom': 80,"marginTop":0,'backgroundColor':'#1167b1'},className='four columns')


### Bans

In [241]:
area1=html.Div([dcc.Graph(id="BAN1")],className='two columns',style={'backgroundColor':'#1167b1','marginBottom': 10,'marginTop': 10})
area2=html.Div([dcc.Graph(id="BAN2",figure={})],className='two columns',style={'backgroundColor':'#1167b1','marginBottom': 10,'marginTop': 10})
area3=html.Div([dcc.Graph(id="BAN3",figure={})],className='two columns',style={'backgroundColor':'#1167b1','marginBottom': 10,'marginTop': 10})
area4=html.Div([dcc.Graph(id="BAN4",figure={})],className='two columns',style={'backgroundColor':'#1167b1','marginBottom': 10,'marginTop': 10})
areas=html.Div([area1,area2,area3,area4],className='row',style={'backgroundColor':'#1167b1'})
head_div=html.Div([header,areas],className='row',style={'backgroundColor':'#1167b1'})


In [242]:
space=html.Div([html.Br()],className='row',style={'backgroundColor':'#1167b1'})

###  1st Tab

In [243]:
tab_selected_style = {
    "background": "white",
    'color': '#1167b1',
    'font-weight':'bold',
    'font-size':22,
    'border-style': 'solid',
    'border-color': 'white',
    'font-family':'Times New Roman'
}
tab_style = {
    "background": "#1167b1",
    'color': 'white',
    'font-size':22,
    'font-weight':'bold',
    'border-style': 'solid',
    'border-color': '#1167b1',
    'font-family':'Times New Roman'
}
tabs_styles = { 'display': 'inlineBlock',
                "background": '#1167b1',
               'border': 'white'}

### Map

In [244]:
map_world=html.Div([html.H1('Countries on track to have vaccinated 40% of their population',
                             style={'font-family':'Times New Roman','font-size':30,'backgroundColor':'#1167b1','color':'white','textAlign':'centre'}),dcc.Dropdown(id="dropdown",
                        options=[
            {'label': 'Africa', 'value': 0},
            {'label': 'Europe', 'value': 1},
            {'label': 'Asia', 'value': 2},
            {'label': 'North America', 'value': 3},
            {'label': 'South America', 'value': 4},
        ],
                        value=None,
                        placeholder="Choose a Continent ......")
    ,dcc.Graph(id="choropleth")
],className='six columns',style={'backgroundColor':'#1167b1'})


### Bar plots

In [245]:
graph1=html.Div([dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Top five countries have people get vaccined', value='tab-1',style=tab_style,selected_style=tab_selected_style),
        dcc.Tab(label='Egypt', value='tab-2',style=tab_style,selected_style=tab_selected_style),
    ],style=tabs_styles),
    html.Div(id='tabs-content')
],className="six columns",style={'backgroundColor':'#1167b1'})

In [246]:
graph_row_1=html.Div([graph1,map_world],className='row',style={'backgroundColor':'#1167b1'})

### 2nd Tab

In [247]:
tap2=html.Div([dcc.Tabs(id="tabs2", value='tab-1', children=[
        dcc.Tab(label='World Wide', value='tab-1',style=tab_style,selected_style=tab_selected_style),
        dcc.Tab(label='Egypt', value='tab-2',style=tab_style,selected_style=tab_selected_style),
    ],style=tabs_styles),
    html.Div(id='tabs-content2')
],className="row",style={'backgroundColor':'#1167b1'})

### slider

#### line graphs

In [248]:
date_slider=html.Div([dcc.RangeSlider(
                         id='date_slider',
                         min=0,
                         max=21,
                         value=[0,21],
    step=1,
marks={
        0: {'label': '2020-01', 'style': {'color': 'white'}},
        1: {'label': '2020-02', 'style': {'color': 'white'}},
        2: {'label': '2020-03', 'style': {'color': 'white'}},
        3: {'label': '2020-04', 'style': {'color': 'white'}},
        4: {'label': '2020-05', 'style': {'color': 'white'}},
        5: {'label': '2020-06', 'style': {'color': 'white'}},
        6: {'label': '2020-07', 'style': {'color': 'white'}},
        7: {'label': '2020-08', 'style': {'color': 'white'}},
        8: {'label': '2020-09', 'style': {'color': 'white'}},
        9: {'label': '2020-10', 'style': {'color': 'white'}},
        10: {'label': '2020-11', 'style': {'color': 'white'}},
        11: {'label': '2020-12', 'style': {'color': 'white'}},
        12: {'label': '2021-01', 'style': {'color': 'white'}},
        13: {'label': '2021-02', 'style': {'color': 'white'}},
        14: {'label': '2021-03', 'style': {'color': 'white'}},
        15: {'label': '2021-04', 'style': {'color': 'white'}},
        16: {'label': '2021-05', 'style': {'color': 'white'}},
        17: {'label': '2021-06', 'style': {'color': 'white'}},
        18: {'label': '2021-07', 'style': {'color': 'white'}},
        19: {'label': '2021-08', 'style': {'color': 'white'}},
        20: {'label': '2021-09', 'style': {'color': 'white'}},
        21: {'label': '2021-10', 'style': {'color': 'white'}},
  },    
    tooltip={"placement": "top"},
#     included=False,
)],className="row")

### 1st CallBack

#### Map and Bans

In [249]:
@app.callback(

    Output("BAN1", "figure"),
    Output("BAN2", "figure"),
    Output("BAN3", "figure"),
    Output("BAN4", "figure"),
    Output("choropleth", "figure"),
    Input("dropdown", "value")
)
def call_bak_update(dropdown):
    fig2=indicator(1)
    fig3=indicator(2)
    fig4=indicator(3)
    fig5=indicator(4)
    fig1=map_fig(dropdown)
    return fig2,fig3,fig4,fig5,fig1

def map_fig(dropdown):
    if dropdown == None:
        fig = px.choropleth(df,
                   locations="Code",
                   color="status",
                color_discrete_map={'Not on track to 40% vaccinated':'#d0efff', 'On track to 40% vaccinated':'#2a9df4','Already above 40% vaccinated':'#03254c'},
#                     color_discrete_sequence="status",
                  hover_name="Entity",
             
                         )
#         fig.update_layout(margin=dict(l=60, r=60, t=50, b=50))
        fig.update_layout(
#         autosize=False,
        margin = dict(
                l=10,
                r=0,
                b=0,
                t=0,
                pad=0,
                autoexpand=True
            ),
#             width=830,
            height=400,
    )
        return fig
    elif dropdown == 0:
        fig = px.choropleth(df,
                   locations="Code",
                   color="status",
                color_discrete_map={'Not on track to 40% vaccinated':'#d0efff', 'On track to 40% vaccinated':'#2a9df4','Already above 40% vaccinated':'#03254c'},
#                     color_discrete_sequence="status",
                  hover_name="Entity",
                   scope="africa" )
        fig.update_layout(
#         autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=0,
                pad=0,
                autoexpand=True
            ),
#             width=830,
            height=400,
        )
        return fig
    elif dropdown == 1:
        fig = px.choropleth(df,
                   locations="Code",
                   color="status",
                color_discrete_map={'Not on track to 40% vaccinated':'#d0efff', 'On track to 40% vaccinated':'#2a9df4','Already above 40% vaccinated':'#03254c'},
#                     color_discrete_sequence="status",
                  hover_name="Entity",
                   scope="europe" )
        fig.update_layout(
#         autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=0,
                pad=0,
                autoexpand=True
            ),
#             width=830,
           height=400,
    )
       
        return fig
    elif dropdown == 2:
        fig = px.choropleth(df,
                   locations="Code",
                   color="status",
                color_discrete_map={'Not on track to 40% vaccinated':'#d0efff', 'On track to 40% vaccinated':'#2a9df4','Already above 40% vaccinated':'#03254c'},
#                     color_discrete_sequence="status",
                  hover_name="Entity",
                   scope="asia" )
        fig.update_layout(
#         autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=0,
                pad=0,
                autoexpand=True
            ),
#             width=830,
            height=400,
    )
        
        return fig
    elif dropdown == 3:
        fig = px.choropleth(df,
                   locations="Code",
                   color="status",
                color_discrete_map={'Not on track to 40% vaccinated':'#d0efff', 'On track to 40% vaccinated':'#2a9df4','Already above 40% vaccinated':'#03254c'},
#                     color_discrete_sequence="status",
                  hover_name="Entity",
                   scope="north america" )
        fig.update_layout(
#         autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=0,
                pad=0,
                autoexpand=True
            ),
#             width=830,
            height=400,
    )
        
        return fig
    elif dropdown == 4:
        fig = px.choropleth(df,
                   locations="Code",
                   color="status",
                color_discrete_map={'Not on track to 40% vaccinated':'#d0efff', 'On track to 40% vaccinated':'#2a9df4','Already above 40% vaccinated':'#03254c'},
#                     color_discrete_sequence="status",
                  hover_name="Entity",
                   scope="south america" )
        fig.update_layout(
#         autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=0,
                pad=0,
                autoexpand=True
            ),
#             width=830,
            height=400,
    )
        return fig


### indicator for Bans

In [250]:
def indicator(x):
    if x==1:
        fig = go.Figure()

        fig.add_trace(go.Indicator(
        mode = "number",
        value = float(round((df2[(df2["location"] == "World")&(df2["Date"]== "2021-10-31")].total_cases)/1000000,2)),
        title = {"text": "<span style='font-size:4mm;color:white'>Total Cases<br>"},
        number={"suffix":"M"}
        ))
        fig.update_layout(
        
        autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=40,
                pad=0,
                autoexpand=False
            ),
            width=250,
      
            height=180,
    
        )
        fig.update_traces(number_font_color="white",selector=dict(type='indicator'))
        fig.update_layout(paper_bgcolor="#1167b1")
    
        return fig
    elif x==2:
        fig = go.Figure()

        fig.add_trace(go.Indicator(
        mode = "number",
        value = float(round((df2[(df2["location"] == "World")&(df2["Date"]== "2021-10-31")].total_deaths)/1000000,2)),
        title = {"text": "<span style='font-size:4mm;color:white'>Total Deaths<br>"},
            number={"suffix":"M"}
        ))
        fig.update_layout(
        
        autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=40,
                pad=0,
                autoexpand=False
            ),
            width=250,
      
            height=180,
    
    )
        fig.update_traces(number_font_color="white",selector=dict(type='indicator'))
        fig.update_layout(paper_bgcolor="#1167b1")
    
        return fig
    elif x==3:
        fig = go.Figure()

        fig.add_trace(go.Indicator(
        mode = "number",
        value = float(round((df2[(df2["location"] == "World")&(df2["Date"]== "2021-10-31")].people_vaccinated)/1000000000,2)),
        title = {"text": "<span style='font-size:4mm;color:white'>At least One dose"},
            number={"suffix":"B"}
        ))
        fig.update_layout(
        
        autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=40,
                pad=0,
                autoexpand=False
            ),
            width=250,
      
            height=180,
    
    )
        fig.update_traces(number_font_color="white",selector=dict(type='indicator'))
        fig.update_layout(paper_bgcolor="#1167b1")
    
        return fig
    else:
      
        fig = go.Figure()

        fig.add_trace(go.Indicator(
        mode = "number",
        value = float(round((df2[(df2["location"] == "World")&(df2["Date"]== "2021-10-31")].people_fully_vaccinated)/1000000000,2)),
        title = {"text": "<span style='font-size:4mm;color:white'>Fully vaccinated"},
            number={"suffix":"B"}
        ))
        fig.update_layout(
        
        autosize=False,
        margin = dict(
                l=0,
                r=0,
                b=0,
                t=40,
                pad=0,
                autoexpand=False
            ),
            width=250,
      
            height=180)
        fig.update_traces(number_font_color="white",selector=dict(type='indicator'))
        fig.update_layout(paper_bgcolor="#1167b1")
    
        return fig

### 2nd CallBack

#### Bar plots

In [251]:
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'),
              )
def first_graph(tab):
    if tab == 'tab-1':
        fig = go.Figure()
        fig.add_trace(go.Bar(
        y=x_5.index,
        x=x_5["people_vaccinated"]- x_5["people_fully_vaccinated"],
        name= 'vaccinated_with_1Dose',
        orientation='h',
        marker=dict(
        color='#03254c',
        line=dict(color='#03254c', width=3)
    )
))
        fig.add_trace(go.Bar(
    y=x_5.index,
    x=x_5["people_fully_vaccinated"],
    name ='fully_vaccinated',
    orientation='h',
    marker=dict(
        color='#187bcd',
        line=dict(color='#187bcd', width=3)
    )
))

        fig.update_layout(barmode='group')
        fig.update_layout(
            height=377
    )
        return html.Div(dcc.Graph(id='graph11',figure=fig))
    
    elif tab == 'tab-2':
        fig2 = go.Figure()
        fig2.add_trace(go.Bar(
    y=x_E["location"],
    x=(x_E["people_vaccinated"] - x_E["people_fully_vaccinated"]),
    name='vaccinated_with_1Dose',
    orientation='h',
    marker=dict(
        color='#03254c',
        line=dict(color='#03254c', width=3)
    )
))
        fig2.add_trace(go.Bar(
    y=x_E["location"],
    x=x_E["people_fully_vaccinated"],
    name='fully_vaccinated',
    orientation='h',
    marker=dict(
        color='#187bcd',
        line=dict(color='#187bcd', width=3)
    )
))

        fig2.update_layout(barmode='group')
        fig2.update_layout(
            height=380
    )
        return html.Div(dcc.Graph(id='graph12',figure=fig2))

### 3rd CallBack

#### line graphs

In [252]:
@app.callback(Output('tabs-content2', 'children'),
              Input('tabs2', 'value'),
              Input("date_slider","value")
              )
def first_graph(tab,date):
    if tab == 'tab-1':
        x = df2[(df2["location"]=="World")&(df2["Date"] >= date_list[date[0]])&(df2["Date"]<=date_list[date[1]])]
        x_1 = x[~(x["new_cases"].isna())]
        fig = px.line(x_1, x="Date", y="new_cases",labels=["Date","new_cases"])
#         fig.update_xaxes(ran)
#         fig.update_yaxes()
        x_2 = x[~(x["new_deaths"].isna())]
        fig2 = px.line(x_2, x="Date", y="new_deaths")
        x_3 = x[~(x["people_vaccinated"].isna())]
        fig3 = px.line(x_3, x="Date", y="people_vaccinated")
        return html.Div([html.Div(dcc.Graph(id='graph20',figure=fig),className="four columns"),html.Div(dcc.Graph(id='graph21',figure=fig2),className="four columns"),html.Div(dcc.Graph(id='graph22',figure=fig3),className="four columns")],style={'backgroundColor':'white'},className="row")
    
    elif tab == 'tab-2':
        x_Eg = df2[(df2["location"] == "Egypt")&(df2["Date"] >= date_list[date[0]])&(df2["Date"]<=date_list[date[1]])]
        x_Eg1 = x_Eg[~(x_Eg["new_cases"].isna())]
        fig4 = px.line(x_Eg1, x="Date", y="new_cases")
        x_Eg3 = x_Eg[~(x_Eg["new_deaths"].isna())]
        fig5 = px.line(x_Eg3, x="Date", y="new_deaths")
        x_Eg2 = x_Eg[~(x_Eg["people_vaccinated"].isna())]
        fig6 = px.line(x_Eg2, x="Date", y="people_vaccinated")
        return  html.Div([html.Div(dcc.Graph(id='graph23',figure=fig4),className="four columns"),html.Div(dcc.Graph(id='graph24',figure=fig5),className="four columns"),html.Div(dcc.Graph(id='graph25',figure=fig6),className="four columns")],style={'backgroundColor':'white'},className="row")

In [253]:
app.layout=html.Div([head_div,space,graph_row_1,space,date_slider,tap2],style={'backgroundColor':'#1167b1'})


#### Run Dashboard

In [254]:
app.run_server(port="8080")

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2021 09:18:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 09:18:44] "POST /_dash-upd